In [8]:
import numpy as np
from json_tricks import dumps, loads
from tqdm import tqdm
import pandas as pd

import torch
import torch.nn.functional as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
from sklearn.model_selection import train_test_split

import pickle
import socket
from datetime import datetime
import random

import gym
from gym.envs.registration import register, spec
from gym import envs

from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, save_model
from keras.optimizers import Adam
import keras.backend as K

from keras.models import load_model

In [9]:
i = 0
lines = []
# with open("data/data4_valid/fl4-comb-new.txt") as f:
with open("data/data4_valid_mul5/fl4-mul5-comb-new.txt") as f:
    for line in f:
        lines.append(line)

In [10]:
Qs = []
str_combs = []
alls = {}
for i in tqdm(range(len(lines))):
    map_str_comb, dic = lines[i].split("\t")
    obj = loads(dic)
    alls[map_str_comb] = obj['Q']
    Qs.append(obj['Q'])
    str_combs.append(map_str_comb)

100%|██████████| 3828/3828 [00:20<00:00, 186.56it/s]


In [11]:
q_data = np.array([q for q in Qs])

In [12]:
X_train, X_test = train_test_split(q_data, test_size=0.10, random_state=42)
X_train_str, X_test_str = train_test_split(str_combs, test_size=0.10, random_state=42)

In [13]:
# G_params = pickle.load(open("data/gan/G_params.p", "rb"))
# D_params = pickle.load(open("data/gan/D_params.p", "rb"))

discriminator = load_model("data/gan/D_params_keras.p")

/Users/gvsi/anaconda3/envs/gan/lib/python3.6/site-packages/keras/models.py:291: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [14]:
np.expand_dims(X_test[0], axis=3).shape

/Users/gvsi/anaconda3/envs/gan/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  """Entry point for launching an IPython kernel.


(16, 4, 1)

In [16]:
X_test2 = np.expand_dims(X_test, axis=3)

In [ ]:
Wzh, bzh, Whx, bhx = G_params
Wxh, bxh, Why, bhy = D_params

In [ ]:
mb_size = 64
ones_label = Variable(torch.ones(mb_size))

In [ ]:
def D(X):
    h = nn.relu(X @ Wxh + bxh.repeat(X.size(0), 1))
    y = nn.sigmoid(h @ Why + bhy.repeat(h.size(0), 1))
    return y

In [ ]:
def get_discriminator_score(Q):
    if len(Q.shape) != 1:
        Q = Q.flatten()
    inputs = Variable(torch.from_numpy(Q).float(), requires_grad=True)
    myD = D(inputs)
    return myD.data.numpy()[0][0]

In [ ]:
def compute_gradients(Q):
    if len(Q.shape) != 1:
        Q = Q.flatten()
    inputs = Variable(torch.from_numpy(Q).float(), requires_grad=True)
    myD = D(inputs)
    myD.backward(ones_label)
    return inputs.grad.data.numpy()

In [12]:
compute_gradients(np.zeros(64))

array([ 1.33704624e+01,  2.23418465e+01, -2.79528713e+00,  3.64787388e+00,
       -3.98031235e-01,  3.28875999e+01, -4.82095337e+01, -2.14143658e+01,
       -9.34957581e+02, -1.00309332e+03, -6.53250793e+02, -1.49343124e+02,
       -5.58164673e+01, -6.09358139e+01, -1.06908310e+02, -8.02173615e-02,
       -2.56729050e+01, -1.74180365e+00, -5.37576103e+01, -3.72171822e+01,
       -1.54025284e+02,  3.98059654e+00,  3.78225174e+01, -1.50335560e+01,
       -8.32711670e+02, -7.80168335e+02, -2.62103455e+02, -8.77553589e+02,
       -5.18191681e+01, -2.07699852e+01,  5.84339619e-01, -7.88252335e+01,
       -1.22757645e+02, -3.13745911e+02, -3.43434906e+02, -5.72222672e+01,
       -6.97571655e+02, -9.58487793e+02, -3.73693268e+02, -4.26875275e+02,
       -4.57048950e+02, -7.45678425e+00, -5.32991333e+02,  2.39936113e+00,
       -1.94286194e+02, -2.81333984e+02, -2.43350159e+02, -1.94121796e+02,
        5.09861279e+00,  3.33634949e+01,  5.99459648e+00,  2.07424393e+01,
       -3.10667358e+02, -

In [13]:
X_test_str[68]

'SHHFFFFFHFHFFFHG'

In [23]:
output = discriminator.outputs[-1]
loss = (1 - output) ** 2
grad = K.gradients(loss, discriminator.inputs[-1])
input_tensors = [discriminator.inputs[0]]
get_gradient = K.function(inputs=input_tensors, outputs=grad)

In [18]:
def new_env(env_map, slippery=True, MY_ENV_NAME='FrozenLakeNonskid-v0'):
    if MY_ENV_NAME in envs.registry.env_specs:
        envs.registry.env_specs.pop(MY_ENV_NAME)

    register(
        id=MY_ENV_NAME,
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'is_slippery': slippery, 'desc': env_map},
        max_episode_steps=100,
        reward_threshold=0.78, # optimum = .8196
    )
    env = gym.make(MY_ENV_NAME)
    return env

In [19]:
def verify(env, Q, num_episodes = 10000):
    print("Running validation...")
    # Set learning parameters
    #create lists to contain total rewards and steps per episode
    #jList = []
    rList = []
    successes = 0
    jTot = 0
    for i in tqdm(range(num_episodes)):
        #Reset environment and get first new observation
        s = env.reset()
        rAll = 0
        d = False
        j = 0
        #The Q-Table learning algorithm
        while j < 200:
            j+=1
            #Choose an action by greedily (with noise) picking from Q table
            a = np.argmax(Q[s,:])
            #Get new state and reward from environment
            s1,r,d,_ = env.step(a)
            rAll += r
            s = s1
            if d == True and r > 0:
                jTot += j
                successes += 1
            if d == True:
                break
        rList.append(rAll)
    print("Score over time: " +  str(sum(rList)/num_episodes))
    valid_score = sum(rList)/num_episodes
    try:
        avg_steps = jTot / successes
    except:
        avg_steps = 0
    return valid_score, avg_steps

In [438]:
class Experiment(object):
    def __init__(self, env, num_episodes=10000):
        self.env = env
        self.Q = np.zeros([self.env.observation_space.n, self.env.action_space.n])
        self.machine = socket.gethostname()
        self.num_episodes = num_episodes
        self.done = False
        self.score = None
        self.start = None
        self.end = None
        self.train_successes = []
        self.valid_score = None
        self.valid_avg_steps = None

    def print_score(self):
        if not self.done:
            print("Run first.")
            return
        print("Score over time: " +  str(self.score))

    def run(self):
        print("Running experiment...")
        if self.done:
            print("Already done running")
            return

        self.start = datetime.now()
        lr = .8
        lr_gan = 0.001
        e = 0.1
        y = .95
        #create lists to contain total rewards and steps per episode
        jList = []
        rList = []
        su1 = 0
        su_tot = 0
        
        for i in tqdm(range(self.num_episodes)):
            #Reset environment and get first new observation
            s = self.env.reset()
            rAll = 0
            d = False
            j = 0
            #The Q-Table learning algorithm
            while j < 200:
                j+=1
                #Choose an action by greedily (with noise) picking from Q table
                a = None
                if random.uniform(0,1) < e:
                    a = self.env.action_space.sample()
                else:
                    a = np.argmax(self.Q[s,:])
                #Get new state and reward from environment
                s1,r,d,_ = self.env.step(a)
                if d == True and r != 1:
                    self.Q[s, a] -= 0.01
                self.Q[s,a] = self.Q[s,a] + lr*(r + y*np.max(self.Q[s1,:]) - self.Q[s,a])
#                 print(discriminator.predict(np.expand_dims(np.array([self.Q]), axis=3)))
#                 print(np.expand_dims(self.Q, axis=3))
                #Update Q-Table with new knowledge
                if i < 500:
                    self.Q[s,a] = self.Q[s,a] + lr*(r + y*np.max(self.Q[s1,:]) - self.Q[s,a])
                    self.Q[s,a] = self.Q[s,a] - 0.5 * np.reshape(get_gradient([np.array([np.expand_dims(self.Q, axis=3)])]), (16, 4))[s,a]
                else:
#                     if np.sign((r + y*np.max(self.Q[s1,:]) - self.Q[s,a])) == np.sign(0.5 * np.reshape(get_gradient([np.array([np.expand_dims(self.Q, axis=3)])]), (16, 4))[s,a]):
#                         su1 += 1
#                     su_tot += 1
#                     old_Q = self.Q
#                     old_Q[s,a] = self.Q[s,a] + lr*(r + y*np.max(self.Q[s1,:]) - self.Q[s,a])
#                     self.Q[s,a] = self.Q[s,a] + get_discriminator_score(old_Q)*(r + y*np.max(self.Q[s1,:]) - self.Q[s,a])
#                     self.Q[s,a] = self.Q[s,a] + lr*(r + y*np.max(self.Q[s1,:]) - self.Q[s,a])
#                     self.Q = self.Q + lr_gan * compute_gradients(self.Q).reshape((self.env.observation_space.n, self.env.action_space.n))
                    self.Q[s,a] = self.Q[s,a] + lr*(r + y*np.max(self.Q[s1,:]) - self.Q[s,a])
#                     print(get_gradient([np.array([np.expand_dims(self.Q, axis=3)])]))
#                     self.Q[s,a] = self.Q[s,a] - 0.5 * np.reshape(get_gradient([np.array([np.expand_dims(self.Q, axis=3)])]), (16, 4))[s,a]
#                     print((r + y*np.max(self.Q[s1,:]) - self.Q[s,a]), np.reshape(get_gradient([np.array([np.expand_dims(self.Q, axis=3)])]), (16, 4))[s,a])
#                     print(np.sign((r + y*np.max(self.Q[s1,:]) - self.Q[s,a])), np.sign(0.5 * np.reshape(get_gradient([np.array([np.expand_dims(self.Q, axis=3)])]), (16, 4))[s,a]))
    
                rAll += r
                s = s1
                if d == True and r > 0:
                    self.train_successes.append((i, j))
                if d == True:
                    #Reduce chance of random action as we train the model.
        #             e = 1./((i/50) + 10)
                    break
            rList.append(rAll)
#         print(su1/su_tot)
        self.done = True
        self.end = datetime.now()
        self.score = sum(rList)/self.num_episodes

    def validate(self):
        valid_score, avg_steps = verify(self.env, self.Q)
        self.valid_score = valid_score
        self.valid_avg_steps = avg_steps

    def dumps(self):
        if not self.done:
            print("Run first.")
            return

        return dumps({'Q': self.Q,
                      'start': self.start,
                      'end': self.end,
                      'train_score': self.score,
                      'num_episodes': self.num_episodes,
                      'train_successes': exp.train_successes,
                      'train_machine' : self.machine,
                      'valid_score' : self.valid_score,
                      'valid_avg_steps' : self.valid_avg_steps
                      })

In [439]:
line = X_train_str[67]
n = 4
map_str_split = [line[i:i+n] for i in range(0, len(line), n)]
env = new_env(map_str_split)

In [440]:
env.render()
exp = Experiment(env, num_episodes=3000)
exp.run()

  1%|          | 21/3000 [00:00<00:15, 189.04it/s]


SFFF
FHHF
HFFF
HFHG
Running experiment...


100%|██████████| 3000/3000 [00:03<00:00, 862.15it/s]


In [441]:
exp.Q

array([[ 3.01665078e-01,  3.19791822e-01,  1.04977438e-01,
         2.96964823e-01],
       [ 1.63992919e-02,  2.13699419e-02,  1.76355078e-02,
         3.75075462e-01],
       [ 5.73409099e-01,  1.84459168e-02,  2.68229480e-02,
         5.44371782e-01],
       [ 4.10917253e-01,  7.56062812e-01,  3.08908570e-01,
         3.17230141e-01],
       [-4.16665796e-04, -4.16610657e-04, -4.16165312e-04,
         1.29921919e-02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 8.10509498e-01,  9.35591386e-01,  7.08317648e-01,
         8.31503131e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [-4.16000000e-04, -4.00000000e-04, -3.08005102e-04,
        -4.00000000e-04],
       [-4.16665600e-04, -4.16453895e-04,  8.68860142e-01,
        -4.16660560e-04],
       [ 7.28784241e-02,  8.83438322e-01,  9.96484344e-01,
      

In [442]:
exp.score

0.13833333333333334

In [443]:
exp.validate()

  7%|▋         | 658/10000 [00:00<00:01, 6567.40it/s]

Running validation...


100%|██████████| 10000/10000 [00:01<00:00, 7517.80it/s]

Score over time: 0.0
